# Studying a set of tracks

In [ ]:
import huracanpy
import numpy as np
import seaborn as sns

## Load tracks
Here we show an example with a CSV file that is embedded within HuracanPy.
HuracanPy supports many track file formats, see [huracanpy.load guide](../load.ipynb) for more details.

Load the ERA5 1996 TC tracks with `huracanpy.load`. 
These are tracks detected by TempestExtremes in ERA5 for the year 1996 and are embedded within HuracanPy as an example.
Here the file extension is '.csv', the function will automatically recognise how to open it.

The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding to each point.
Variables indicate position in space and time, as well as additional attributes such as maximum wind speed and minimum slp.

In [ ]:
file = huracanpy.example_year_file
print(file.split("/")[-1])

tracks = huracanpy.load(file)
tracks

## Adding info to the tracks
HuracanPy has several function to add useful information to the tracks (for full list, see [huracanpy.info](../../api/info.rst)). Here for example we add basin and Saffir-Simpson hurrican scale category information.]

### Add basin

In [ ]:
tracks = tracks.hrcn.add_basin()
tracks.basin

#### Show distribution of TC points among basins
(calling seaborn function, works better with categorical labels)

In [ ]:
sns.countplot(tracks.basin)

### Add Saffir-Simpson and pressure-based categories
Note: in ERA5 data, wind is stored in wind10 in m/s

In [ ]:
tracks = tracks.hrcn.add_saffir_simpson_category(wind_name="wind10", wind_units="m s-1")
tracks = tracks.hrcn.add_pressure_category(slp_name="slp", slp_units="Pa")
tracks[["saffir_simpson_category", "pressure_category"]]

#### Show distribution of TC points among categories
(using xarray's built-in function)

In [ ]:
bins = np.arange(-1.5, 5.5 + 1)
tracks.saffir_simpson_category.plot.hist(bins=bins, alpha=0.5)
tracks.pressure_category.plot.hist(bins=bins, alpha=0.5)

## Plotting
HuracanPy embeds basic plotting functions, which are mainly meant for having a preliminary look at your data. In particular here we show how to plot the track points themselves, and track density. The [example gallery](../../examples/index.rst) displays nice plots made from HuracanPy and the associated scripts.
### Plotting the tracks
Plot ERA5 tracks colored by wind intensity

In [ ]:
tracks.hrcn.plot_tracks(intensity_var_name="wind10")

### Plotting track density
You can plot the track density directly with `plot_density`, which is based on a simple 2D histogram of TC points

In [ ]:
tracks.hrcn.plot_density()

You can also get the underlying density matrix with `get_density` and then use it to make you own plots in your favourite way

In [ ]:
tracks.hrcn.get_density()

### Plotting genesis points
`get_gen_vals` allows you to subset only the genesis points in an efficient way

In [ ]:
gen_points = tracks.hrcn.get_gen_vals()
gen_points

If you use `plot_tracks` on these, you can display only the genesis points.

In [ ]:
gen_points.hrcn.plot_tracks()

## Compute statistics

### Number of cyclones
Count number of unique track ids

In [ ]:
tracks.track_id.hrcn.nunique()

### Cyclone duration & TC days
Get the duration for each track.
The result is an `xarray.Dataset` with "track_id" as a coordinate

In [ ]:
TC_duration = tracks.hrcn.get_track_duration()
TC_duration

Compute the total number of TC days.
Sum all the durations (and divide by 24 because durations are in hours)

In [ ]:
TC_duration.sum() / 24

### Cyclone Intensity

There are two ways to obtain the lifetime maximum intensity (LMI) of each tracks

1. Use `get_apex_vals`, which returns the subset of points only at specified LMI

In [ ]:
lmi_points = tracks.hrcn.get_apex_vals(var_name="wind10")
lmi_points

2. Compute lifetime maximum intensity per track with xarray's `groupby`

In [ ]:
lmi_wind = tracks.wind10.groupby(tracks.track_id).max()
lmi_wind

You can then plot the LMI distribution using xarray's built-in plot function.

In [ ]:
lmi_wind.plot.hist()

### ACE
Accumulated cyclone energy (ACE) is a commonly used measure of cyclone activity that combines the energy and duration of cyclones.

#### Compute ACE for each individual track point

In [ ]:
tracks = tracks.hrcn.add_ace(wind_name="wind10", wind_units="m s-1")
tracks.ace

#### Compute total ACE

In [ ]:
tracks.ace.sum()

## Compositing lifecycle
Add time from apex variable to be able to superimpose all the tracks centered on apex.
Here we use minimum pressure as the apex point

### Add time from minimum pressure

In [ ]:
tracks = tracks.hrcn.add_time_from_apex(intensity_var_name="slp", stat="min")
tracks.time_from_apex

In [ ]:
tracks.time_from_apex / np.timedelta64(1, "h")

### Plot composite SLP lifecycle

In [ ]:
# Convert time_from_apex to hours
tracks["time_from_apex"] = tracks.time_from_apex / np.timedelta64(1, "h")

# Use xarray's where to mask points too far away from apex (48 hours away)
tracks_close_to_apex = tracks.where(np.abs(tracks.time_from_apex) <= 48, drop=True)

# Seaborn lineplot allows for drawing composites with uncertainty range
# x-axis is time from apex
# y-axis is slp, converted to hPa
sns.lineplot(x=tracks_close_to_apex.time_from_apex, y=tracks_close_to_apex.slp / 100)